In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import sqlite3 as sql
from preprocess import DB
import matplotlib.pyplot as plt

In [2]:
db = DB(db_route = '../data/database.sqlite')

In [3]:
df = db.combineCardFoulDF()

  0%|          | 0/14217 [00:00<?, ?it/s]

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [29]:
for r in root[2]:
    print(r.tag, r.text)

comment y
stats None
elapsed_plus 4
event_incident_typefk 70
elapsed 45
card_type y
player1 24846
sortorder 1
team 8668
n 221
type card
id 2139080


In [4]:
db.unravelCardDF(36)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [5]:
index = 36
xml = db.match_team['card'].iloc[index]
match_id = db.match_team['match_api_id'].iloc[index]
root = etree.fromstring(xml)

keys_d = [
    'elapsed','match_id', 'id','foulscommitted','event_incident_typefk','player1',
    'player2','team','type','subtype','ycards','rcards','comment', 'n', 'sortorder'
]
unique_elems = []
for child in root:
    if child[0].tag not in unique_elems and child[0].tag not in keys_d:
        unique_elems += [child[0].tag]
    if child[1].tag == 'stats' and child[1][0].tag not in unique_elems and child[1][0].tag not in keys_d:
        unique_elems += [child[1][0].tag]
    elif child[1].tag != 'stats':
        for child1 in child[1:]:
            if child1.tag not in unique_elems and child1.tag not in keys_d and child1.tag != 'del':
                unique_elems += [child1.tag]
        
    for child1 in child[2:]:
        if child1.tag not in unique_elems and child1.tag not in keys_d:
            unique_elems += [child1.tag]

d = {}
for k in keys_d:
    d[k] = []

for elem in unique_elems:
    d[elem] = []

for i, child in enumerate(root):
    d['match_id'] += [match_id]
    d[child[0].tag] += [child[0].text]
    
    if child[1].tag == 'stats':
        d[child[1][0].tag] += [child[1][0].text]

        for child1 in child[2:]:
            d[child1.tag] += [child1.text]
    else:
        for child1 in child[1:]:
            if child1.tag != 'del':
                d[child1.tag] += [child1.text]

    key_lengths = [len(d[x]) for x in list(d.keys())]
    for x in list(d.keys()):
        if len(d[x]) < max(key_lengths):
            d[x] += [None]

card_df = pd.DataFrame(d).drop('del', axis = 1)

In [6]:
card_df

,elapsed,match_id,id,foulscommitted,event_incident_typefk,player1,player2,team,type,subtype,ycards,rcards,comment,n,sortorder,card_type,elapsed_plus
0,7,1026165,2138102,None,70,30839,None,8668,card,None,1,None,y,188,1,y,None
1,10,1026165,2138160,None,70,38373,None,10261,card,None,1,None,y,191,1,y,None
2,45,1026165,2139080,None,70,24846,None,8668,card,None,1,None,y,221,1,y,4
3,90,1026165,2140457,None,270,None,None,None,card,None,None,None,r,164,1,None,5
4,90,1026165,2140467,None,67,30876,None,8668,card,violence,None,1,r,269,1,r,5
